<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Image%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!7z x /content/Dataset.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 17231696 bytes (17 MiB)

Extracting archive: /content/Dataset.7z
--
Path = /content/Dataset.7z
Type = 7z
Physical Size = 17231696
Headers Size = 6422
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     18% 23 - Dataset/all_meta_data.csv                                    35% 144 - Dataset/Dibothriocephalus/Diphyllobothrium_tissue_WA_500x2.jpg                                                                          53% 252 - Dataset/Giardia/Giardia_cyst_tric6.j

# Installing and Importing Packages

In [3]:
!pip install einops -q
import numpy as np
import pandas as pd
from PIL import Image
import time
import torch
from torchvision import transforms
import torch.nn.functional as F
from einops import rearrange
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.models as models

# Vision Bert

In [51]:
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) + x

class LayerNormalize(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class MLP_Block(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.1):
        super().__init__()
        self.nn1 = nn.Linear(dim, hidden_dim)
        torch.nn.init.xavier_uniform_(self.nn1.weight)
        torch.nn.init.normal_(self.nn1.bias, std = 1e-6)
        self.af1 = nn.GELU()
        self.do1 = nn.Dropout(dropout)
        self.nn2 = nn.Linear(hidden_dim, dim)
        torch.nn.init.xavier_uniform_(self.nn2.weight)
        torch.nn.init.normal_(self.nn2.bias, std = 1e-6)
        self.do2 = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.nn1(x)
        x = self.af1(x)
        x = self.do1(x)
        x = self.nn2(x)
        x = self.do2(x)
        
        return x

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dropout = 0.1):
        super().__init__()
        self.heads = heads
        self.scale = dim ** -0.5  # 1/sqrt(dim)

        self.to_qkv = nn.Linear(dim, dim * 3, bias = True) # Wq,Wk,Wv for each vector, thats why *3
        torch.nn.init.xavier_uniform_(self.to_qkv.weight)
        torch.nn.init.zeros_(self.to_qkv.bias)
        
        self.nn1 = nn.Linear(dim, dim)
        torch.nn.init.xavier_uniform_(self.nn1.weight)
        torch.nn.init.zeros_(self.nn1.bias)        
        self.do1 = nn.Dropout(dropout)
        

    def forward(self, x, mask = None):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x) #gets q = Q = Wq matmul x1, k = Wk mm x2, v = Wv mm x3
        q, k, v = rearrange(qkv, 'b n (qkv h d) -> qkv b h n d', qkv = 3, h = h) # split into multi head attentions

        dots = torch.einsum('bhid,bhjd->bhij', q, k) * self.scale

        if mask is not None:
            mask = F.pad(mask.flatten(1), (1, 0), value = True)
            assert mask.shape[-1] == dots.shape[-1], 'mask has incorrect dimensions'
            mask = mask[:, None, :] * mask[:, :, None]
            dots.masked_fill_(~mask, float('-inf'))
            del mask

        attn = dots.softmax(dim=-1) #follow the softmax,q,d,v equation in the paper

        out = torch.einsum('bhij,bhjd->bhid', attn, v) #product of v times whatever inside softmax
        out = rearrange(out, 'b h n d -> b n (h d)') #concat heads into one matrix, ready for next encoder block
        out =  self.nn1(out)
        out = self.do1(out)
        return out

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim, dropout):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Residual(LayerNormalize(dim, Attention(dim, heads = heads, dropout = dropout))),
                Residual(LayerNormalize(dim, MLP_Block(dim, mlp_dim, dropout = dropout)))
            ]))
    def forward(self, x, mask = None):
        for attention, mlp in self.layers:
            x = attention(x, mask = mask) # go to attention
            x = mlp(x) #go to MLP_Block
        return x

class ImageTransformer(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels = 3, dropout = 0.1, emb_dropout = 0.1):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2  # e.g. (32/4)**2= 64
        patch_dim = channels * patch_size ** 2  # e.g. 3*8**2 = 64*3

        self.patch_size = patch_size
        self.pos_embedding = nn.Parameter(torch.empty(1, (num_patches + 1), dim))
        torch.nn.init.normal_(self.pos_embedding, std = .02) # initialized based on the paper
        self.patch_conv= nn.Conv2d(3,dim, patch_size, stride = patch_size) #eqivalent to x matmul E, E= embedd matrix, this is the linear patch projection
        
        #self.E = nn.Parameter(nn.init.normal_(torch.empty(BATCH_SIZE_TRAIN,patch_dim,dim)),requires_grad = True)
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, dim)) #initialized based on the paper
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, mlp_dim, dropout)

        self.to_cls_token = nn.Identity()

        self.nn1 = nn.Linear(dim, num_classes)  # if finetuning, just use a linear layer without further hidden layers (paper)
        torch.nn.init.xavier_uniform_(self.nn1.weight)
        torch.nn.init.normal_(self.nn1.bias, std = 1e-6)
        # self.af1 = nn.GELU() # use additinal hidden layers only when training on large datasets
        # self.do1 = nn.Dropout(dropout)
        # self.nn2 = nn.Linear(mlp_dim, num_classes)
        # torch.nn.init.xavier_uniform_(self.nn2.weight)
        # torch.nn.init.normal_(self.nn2.bias)
        # self.do2 = nn.Dropout(dropout)

    def forward(self, img, mask = None):
        p = self.patch_size

        x = self.patch_conv(img) # each of 64 vecotrs is linearly transformed with a FFN equiv to E matmul
        #x = torch.matmul(x, self.E)
        x = rearrange(x, 'b c h w -> b (h w) c') # 64 vectors in rows representing 64 patches, each 64*3 long

        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        x = self.dropout(x)

        x = self.transformer(x, mask) #main game

        x = self.to_cls_token(x[:, 0])
        
        x = self.nn1(x)
        # x = self.af1(x)
        # x = self.do1(x)
        # x = self.nn2(x)
        # x = self.do2(x)
        
        return x


In [1]:

model = models.vgg19()

# Dataset Loading

In [6]:
## Getting images(file path) from the directories 
import os
imgs = []
for path, subdirs, files in os.walk('./Dataset'):
    for name in files:
        imgs.append(os.path.join(path, name))
imgs = [x for x in imgs if '.jpg' in x]

In [7]:
data = pd.read_csv("/content/Dataset/all_meta_data.csv")
data.head()

,phylum,class,genus,species,form,sample,image_name,image_url,img_path
0,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_worm4_HB.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_worm4_HB.jpg
1,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_HBa.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_HBa.jpg
2,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_wtmt.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_wtmt.jpg
3,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_SC_egg.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_SC_egg.jpg
4,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_UVa.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_UVa.jpg


In [8]:
# Dropping unnecessary datapoints
def check_file(x):
  if x not in imgs:
    return 'N\A'
  else:
    return x


data['img_path'] = data['img_path'].apply(check_file)
data.drop(index = data[data['img_path']=='N\A'].index,inplace = True)

In [9]:
data.to_csv('/content/Parasitesv1.csv',index=False)

## Custom Dataset Loader

In [56]:
from torch.utils.data import Dataset, DataLoader, sampler
from sklearn.preprocessing import LabelEncoder
class SpeciesLoader(Dataset):
  def __init__(self,csv_file,transform):
    super().__init__()
    csv = pd.read_csv(csv_file)[['species','img_path']]
    labels = csv['species'].values

    self.images = csv['img_path'].values
    self.transform = transform

    self.LE = LabelEncoder()
    self.labels = self.LE.fit_transform(labels)    

  def __len__(self):
    # return size of dataset
    return len(self.images)

  def __getitem__(self, index):
    img = Image.open(self.images[index])
    img = self.transform(img)

    label = self.labels[index]

    return img, label

In [60]:
transformer = transforms.Compose([
        transforms.Resize((32,32)),
        transforms.RandomHorizontalFlip(),
        # transforms.
        # transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomRotation(90),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

## Train and Test splitting

In [61]:
dataset = SpeciesLoader('/content/Parasitesv1.csv',transform=transformer)
batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


In [43]:
idk = []
for i, (data, target) in enumerate(train_loader):
  idk+=data

In [45]:
len(train_loader)

24

# Train and Eval Functions

In [67]:
def train(model, optimizer, data_loader, loss_history):
    total_samples = 0
    model.train()

    for i, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        data = data.cuda()
        total_samples += len(data)
        target = target.cuda()
        output = F.log_softmax(model(data), dim=1)
        loss = F.nll_loss(output, target)

        loss.backward()
        optimizer.step()

        if i % 16 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_history.append(loss.item())

def evaluate(model, data_loader, loss_history):
    model.eval()
    
    total_samples = 0
    correct_samples = 0
    total_loss = 0

    with torch.no_grad():
        for data, target in data_loader:
            total_samples += len(data)
            data = data.cuda()
            target = target.cuda()
            output = F.log_softmax(model(data), dim=1)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / total_samples
    loss_history.append(avg_loss)
    print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')

In [ ]:
class VisualNet(nn.Module):
  def __init__(self,trained_weights = True,trainable = False):
    self.vgg = models.vgg19_bn(pretrained = trained)
    self.classifier = nn.Linear

# ImageTransformer

In [52]:
N_EPOCHS = 150

model = ImageTransformer(image_size=32, patch_size=4, num_classes=27, channels=3,
            dim=64, depth=6, heads=8, mlp_dim=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

In [62]:
train_loss_history, test_loss_history = [], []
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    start_time = time.time()
    train(model, optimizer, train_loader, train_loss_history)
    print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')
    evaluate(model, validation_loader, test_loss_history)

print('Execution time')

Epoch: 1
[    0/  382 (  0%)]  Loss: 6.4166
[  256/  382 ( 67%)]  Loss: 5.7726
Execution time:  3.28 seconds

Average test loss: 3.9720  Accuracy:   10/   95 (10.53%)

Epoch: 2
[    0/  382 (  0%)]  Loss: 3.8695
[  256/  382 ( 67%)]  Loss: 4.1303
Execution time:  3.22 seconds

Average test loss: 3.7654  Accuracy:   13/   95 (13.68%)

Epoch: 3
[    0/  382 (  0%)]  Loss: 3.2053
[  256/  382 ( 67%)]  Loss: 3.2161
Execution time:  3.22 seconds

Average test loss: 3.4294  Accuracy:   20/   95 (21.05%)

Epoch: 4
[    0/  382 (  0%)]  Loss: 2.6581
[  256/  382 ( 67%)]  Loss: 3.5478
Execution time:  3.21 seconds

Average test loss: 3.3281  Accuracy:   17/   95 (17.89%)

Epoch: 5
[    0/  382 (  0%)]  Loss: 2.3685
[  256/  382 ( 67%)]  Loss: 3.8867
Execution time:  3.20 seconds

Average test loss: 3.1307  Accuracy:   15/   95 (15.79%)

Epoch: 6
[    0/  382 (  0%)]  Loss: 2.9854
[  256/  382 ( 67%)]  Loss: 2.5924
Execution time:  3.17 seconds

Average test loss: 3.6917  Accuracy:   18/   95 (1

# VGG19-Untrained

In [68]:
N_EPOCHS = 150

model = models.vgg19_bn(num_classes = 27)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [69]:
train_loss_history, test_loss_history = [], []

e_start = time.time()
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    start_time = time.time()
    train(model, optimizer, train_loader, train_loss_history)
    print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')
    evaluate(model, validation_loader, test_loss_history)

print(f'Execution time {time.time()-e_start}')

Epoch: 1
[    0/   16 (  0%)]  Loss: 3.7793
[  256/  272 ( 67%)]  Loss: 3.2558
Execution time:  1.80 seconds

Average test loss: 3.2763  Accuracy:    7/   95 (7.37%)

Epoch: 2
[    0/   16 (  0%)]  Loss: 3.3121
[  256/  272 ( 67%)]  Loss: 2.9432
Execution time:  1.83 seconds

Average test loss: 3.1884  Accuracy:    7/   95 (7.37%)

Epoch: 3
[    0/   16 (  0%)]  Loss: 3.5999
[  256/  272 ( 67%)]  Loss: 3.5572
Execution time:  1.79 seconds

Average test loss: 3.1979  Accuracy:    7/   95 (7.37%)

Epoch: 4
[    0/   16 (  0%)]  Loss: 3.2192
[  256/  272 ( 67%)]  Loss: 3.0254
Execution time:  1.77 seconds

Average test loss: 3.1438  Accuracy:    7/   95 (7.37%)

Epoch: 5
[    0/   16 (  0%)]  Loss: 2.8820
[  256/  272 ( 67%)]  Loss: 3.7351
Execution time:  1.78 seconds

Average test loss: 3.1041  Accuracy:   12/   95 (12.63%)

Epoch: 6
[    0/   16 (  0%)]  Loss: 2.7266
[  256/  272 ( 67%)]  Loss: 3.2384
Execution time:  1.78 seconds

Average test loss: 2.9441  Accuracy:   10/   95 (10.53

# ResNET-Untrained

In [70]:
N_EPOCHS = 150

model = models.resnet50(num_classes = 27)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [71]:
train_loss_history, test_loss_history = [], []

e_start = time.time()
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    start_time = time.time()
    train(model, optimizer, train_loader, train_loss_history)
    print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')
    evaluate(model, validation_loader, test_loss_history)

print(f'Execution time {time.time()-e_start}')

Epoch: 1
[    0/   16 (  0%)]  Loss: 3.3534
[  256/  272 ( 67%)]  Loss: 3.4302
Execution time:  2.54 seconds

Average test loss: 3.3178  Accuracy:    8/   95 (8.42%)

Epoch: 2
[    0/   16 (  0%)]  Loss: 3.4780
[  256/  272 ( 67%)]  Loss: 3.2394
Execution time:  2.51 seconds

Average test loss: 3.3138  Accuracy:    6/   95 (6.32%)

Epoch: 3
[    0/   16 (  0%)]  Loss: 2.7562
[  256/  272 ( 67%)]  Loss: 3.4436
Execution time:  2.51 seconds

Average test loss: 3.4042  Accuracy:    2/   95 (2.11%)

Epoch: 4
[    0/   16 (  0%)]  Loss: 3.3589
[  256/  272 ( 67%)]  Loss: 3.0570
Execution time:  2.54 seconds

Average test loss: 3.4150  Accuracy:    5/   95 (5.26%)

Epoch: 5
[    0/   16 (  0%)]  Loss: 2.8129
[  256/  272 ( 67%)]  Loss: 2.8335
Execution time:  2.56 seconds

Average test loss: 3.3664  Accuracy:    5/   95 (5.26%)

Epoch: 6
[    0/   16 (  0%)]  Loss: 2.9513
[  256/  272 ( 67%)]  Loss: 3.5507
Execution time:  2.50 seconds

Average test loss: 3.3731  Accuracy:    5/   95 (5.26%)